# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [647]:
# %pip install --upgrade pandas

In [648]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수

In [649]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 데이터 불러오기

In [650]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [651]:
train_tr[train_tr['대분류'] == '공산품']

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
6,train_12777,2004-05-01 10:30:00,A112000,공산품,일용잡화,5100,17000
9,train_35,2004-05-01 10:40:00,A112000,공산품,용기보증,5100,28358
14,train_14716,2004-05-01 10:40:00,A373000,공산품,상품군미지정,5100,8100
15,train_7407,2004-05-01 10:40:00,A112000,공산품,야채,5100,9560
...,...,...,...,...,...,...,...
523054,train_10849,2005-04-29 19:20:00,A202000,공산품,냉장식품,5100,3320
523079,train_5332,2005-04-29 19:30:00,A202000,공산품,과자,5100,27000
523093,train_5332,2005-04-29 19:40:00,A202000,공산품,가정잡화,5100,19778
523096,train_13390,2005-04-29 19:50:00,A112000,공산품,용기보증,5100,91826


In [652]:
# from IPython.display import display, HTML
# br_html = HTML("<br><br>")
# display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
# display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
# display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
# display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)

In [653]:
# display(HTML("<h1>학습용 구매기록 데이터</h1>"))
# display(train_tr.info(), br_html)

# display(HTML("<h1>학습용 정답 데이터</h1>"))
# display(train_target.info(), br_html)

# display(HTML("<h1>테스트용 구매기록 데이터</h1>"))
# display(test_tr.info(), br_html)

# display(HTML("<h1>제출 양식 데이터</h1>"))
# display(submit.info(), br_html)

In [654]:
# pd.concat([train_tr, train_target], axis = 1)

# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [655]:
# train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
# test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
# train_tr["구매일시"].max()

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [656]:
# train_ft = train_target[["ID"]]
# test_ft = submit[["ID"]]

# train_ft.shape, test_ft.shape

In [657]:
# train_tr['중분류'].values[:]

## 구매일시를 이용한 특성생성

In [658]:
# 저녁 시간18~21, 5월4일~5월5일, 어버이날

In [659]:
# agg_list = [
#         # 컬럼명, 집계 방식
#         ('내점일수','nunique'),
#         ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
#         ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
#         ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
#         ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
#         ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
#         ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
#         ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
#         ('일별평균구매건수', lambda x:  x.count() / x.dt.date.nunique() ),
#         ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
#     ]

# tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [660]:
# tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 지점을 이용한 특성생성

In [661]:
# agg_list = [
#           ("방문지점수","nunique"),
#           ('주구매지점', lambda x: x.mode()[0]),
#     ]

# tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [662]:
# tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 브랜드코드를 이용한 특성생성

In [663]:
# agg_list = [
#              ('브랜드코드_nunique', 'nunique'),
#              ]

# tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [664]:
# tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 중분류를 이용한 특성생성

In [665]:
# agg_list = [
#             ('중분류_nunique', 'nunique'),
#             ('주구매_중분류', lambda x: x.mode()[0]),
#             ]
# tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [666]:
# tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

## 대분류를 이용한 특성생성

In [667]:
# agg_list = [
#             ('대분류_nunique', 'nunique'),
#             ('주구매_대분류', lambda x: x.mode()[0]),
#             ]

# tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [668]:
# tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

## 구매가격을 이용한 특성생성

In [669]:
# agg_list = [
#         ('총구매액','sum'),
#         ('구매건수', 'count'),
#         ('평균구매액', "mean"),
#         ('최대구매액', 'max'),
#         ('총구매액_왜도', 'skew'),
#         ('최소구매액',lambda x: x[x > 0].min() ) ,
#         ('환불금액',lambda x: x[x < 0].sum() ) ,
#         ('환불건수', lambda x: ( x < 0 ).sum() ),
#         ('구매금액표준편차',lambda x: x[x>0].std() )
#     ]

# tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [670]:
# tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [671]:
# train_ft.shape, test_ft.shape

## pivot_table을 이용한 특성 생성

In [672]:
# train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
# train_ft = train_ft.merge(train_tmp,how="left",on="ID")
# train_ft.head()

In [673]:
# test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
# test_tmp.head()

In [674]:
# for col in train_tmp.columns:
#     if col not in test_tmp.columns:
#         test_tmp[col] = 0

# test_tmp = test_tmp[train_tmp.columns]

In [675]:
# test_ft = test_ft.merge(test_tmp,how="left",on="ID")
# test_ft.head()

In [676]:
# cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

In [677]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

In [678]:
final_project = '/content/drive/MyDrive/멋사_final_project/'

In [679]:
train_ft = pd.read_csv(f"{final_project}train_baseline.csv") # 특성추출_베이스라인 train 결과 불러오기
test_ft = pd.read_csv(f"{final_project}test_baseline.csv") # 특성추출_베이스라인 test 결과 불러오기


train_ft.shape, test_ft.shape

((14940, 332), (12225, 332))

## 구매일시 활용

In [680]:
# 저녁 시간18~21, 5월4일~5월5일(어린이날), 5월8일(어버이날)

In [681]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"], format='%Y-%m-%d %H:%M:%S')
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"], format='%Y-%m-%d %H:%M:%S')

In [682]:
train_tr['구매일시'].dtype

dtype('<M8[ns]')

In [683]:
train_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523105 entries, 0 to 523104
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   ID      523105 non-null  object        
 1   구매일시    523105 non-null  datetime64[ns]
 2   지점코드    523105 non-null  object        
 3   대분류     523105 non-null  object        
 4   중분류     523105 non-null  object        
 5   브랜드코드   523105 non-null  int64         
 6   구매가격    523105 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 27.9+ MB


In [684]:
train_tr['구매일시'].isnull().sum()

0

In [685]:
train_tr['구매일시'].head()

,구매일시
0,2004-05-01 09:40:00
1,2004-05-01 09:40:00
2,2004-05-01 10:20:00
3,2004-05-01 10:30:00
4,2004-05-01 10:30:00


In [686]:
# ((train_tr["구매일시"].dt.hour > 18) & (train_tr["구매일시"].dt.hour < 21)).mean()

In [687]:
# display(train_tr["구매일시"].max())
# display(train_tr["구매일시"].min())

In [688]:
# train_tr["구매일시"].dt.month == 5

In [689]:
# (train_tr["구매일시"].dt.day == 4).value_counts()

In [690]:
# np.mean((train_tr["구매일시"].dt.month == 5) & (train_tr["구매일시"].dt.day == 4))

In [691]:
# 날짜 기준 ID별 5/5 전날 > groupby 구매비용 + 아동 포함된 구매비용

In [692]:
# 기본 구매 횟수 계산
train_tr['hour'] = train_tr['구매일시'].dt.hour
train_tr['date'] = train_tr['구매일시'].dt.date

children_day = pd.to_datetime('2004-05-05').date()
parents_day = pd.to_datetime('2004-05-08').date()

# 시간대 및 기념일 전후 기준 열 생성
train_tr['18시~21시'] = ((train_tr['hour'] >= 18) & (train_tr['hour'] <= 21)).astype(int)
train_tr['어린이날_전날'] = (train_tr['date'] == children_day - pd.Timedelta(days=1)).astype(int)
train_tr['어린이날_당일'] = (train_tr['date'] == children_day).astype(int)
train_tr['어버이날_전날'] = (train_tr['date'] == parents_day - pd.Timedelta(days=1)).astype(int)
train_tr['어버이날_당일'] = (train_tr['date'] == parents_day).astype(int)

# 각 통계 값 계산을 위한 groupby
agg_dict = {
    '구매일시': 'count',
    '18시~21시': ['sum', 'mean'],
    '어린이날_전날': ['sum', 'mean'],
    '어린이날_당일': ['sum', 'mean'],
    '어버이날_전날': ['sum', 'mean'],
    '어버이날_당일': ['sum', 'mean']
}

# groupby 집계 수행
tmp = train_tr.groupby('ID').agg(agg_dict).reset_index()

# 컬럼 이름 정리
tmp.columns = [
    'ID', '구매횟수',
    '18시~21시_구매횟수', '18시~21시_구매비율',
    '어린이날_전날_구매횟수', '어린이날_전날_구매비율',
    '어린이날_당일_구매횟수', '어린이날_당일_구매비율',
    '어버이날_전날_구매횟수', '어버이날_전날_구매비율',
    '어버이날_당일_구매횟수', '어버이날_당일_구매비율'
]

# 최종 병합
train_ft = train_ft.merge(tmp, on="ID", how='left')
test_ft = test_ft.merge(tmp, on="ID", how='left')

In [693]:
train_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,18시~21시_구매횟수,18시~21시_구매비율,어린이날_전날_구매횟수,어린이날_전날_구매비율,어린이날_당일_구매횟수,어린이날_당일_구매비율,어버이날_전날_구매횟수,어버이날_전날_구매비율,어버이날_당일_구매횟수,어버이날_당일_구매비율
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,5,0.250000,0,0.000000,0,0.0,1,0.050000,0,0.0
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,13,0.309524,0,0.000000,0,0.0,0,0.000000,0,0.0
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,17,0.149123,0,0.000000,0,0.0,0,0.000000,0,0.0
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,47,0.222749,0,0.000000,0,0.0,0,0.000000,0,0.0
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,10,0.161290,0,0.000000,0,0.0,0,0.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,3,0.157895,0,0.000000,0,0.0,0,0.000000,0,0.0
14936,train_14936,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,4,0.500000,0,0.000000,0,0.0,0,0.000000,0,0.0
14937,train_14937,22,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,2,0.083333,0,0.000000,0,0.0,0,0.000000,0,0.0
14938,train_14938,31,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,12,0.363636,1,0.030303,0,0.0,0,0.000000,0,0.0


In [694]:
test_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,18시~21시_구매횟수,18시~21시_구매비율,어린이날_전날_구매횟수,어린이날_전날_구매비율,어린이날_당일_구매횟수,어린이날_당일_구매비율,어버이날_전날_구매횟수,어버이날_전날_구매비율,어버이날_당일_구매횟수,어버이날_당일_구매비율
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_12220,12,47,0.000000,0.166667,0.166667,0.250000,0.416667,2,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12221,test_12221,19,24,0.238095,0.380952,0.000000,0.238095,0.380952,0,1.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12222,test_12222,38,12,0.315789,0.157895,0.263158,0.289474,0.289474,2,1.357143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12223,test_12223,221,3,0.202335,0.171206,0.311284,0.287938,0.229572,1,2.234783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [695]:
# # code_test
# agg_list = [
#         # 컬럼명, 집계 방식
#         ('퇴근후_구매비율', lambda x: np.mean((x.dt.hour >= 18) & (x.dt.hour <= 21))),
#         ('2004_어린이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 4))), # 아동관련 구매비율
#         ('2004_어린이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 5))),
#         ('2004_어버이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 7))),
#         ('2004_어버이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 8))),
#     ]
# train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()

In [696]:
# # code_acting
# agg_list = [
#         # 컬럼명, 집계 방식
#         ('퇴근후_구매비율', lambda x: np.mean((x.dt.hour >= 18) & (x.dt.hour <= 21))),
#         ('2004_어린이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 4))),
#         ('2004_어린이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 5))),
#         ('2004_어버이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 7))),
#         ('2004_어버이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 8))),
#     ]

# tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [697]:
# tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

In [698]:
train_ft.shape, test_ft.shape

((14940, 343), (12225, 343))

## 지점 활용

In [699]:
# 전부 기혼 우세

In [700]:
# train_tr['지점코드'].value_counts()

In [701]:
# train_tr['지점코드'].unique()

In [702]:
# #code test
# agg_list = [
#           ('', ),
#     ]
# train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()

In [703]:
# # code_acting
# agg_list = [
#           ('', ),
#     ]

# tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [704]:
# tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 브랜드코드 활용

In [705]:
# 기혼특징
#   5100, 5110, 5111, 5112
# 공산품포함 브랜드
#   5100, 5338

In [706]:
train_tr[train_tr['대분류'] == '공산품']['브랜드코드'].unique()

array([5100, 5338])

In [707]:
train_tr['브랜드코드'].unique()

array([5100, 5101, 5105, ..., 6994, 6996, 6524])

In [708]:
train_tr['브랜드코드'].value_counts()

,count
브랜드코드,
5100,119088
5127,8860
5173,6517
5116,5682
5138,5277
...,...
6894,1
6892,1
6506,1


In [709]:
# train_tr.groupby('ID')["브랜드코드"].count()

In [710]:
# 공산품_포함
agg_list = [
    ('공산품_포함관련구매비용', 'sum'),
    ('공산품_포함관련구매건수', 'count'),
    ('공산품_포함관련구매평균', 'mean')
]

# train 데이터에 대해 공산품_포함관련 데이터 집계
fac_product_related = train_tr[train_tr['브랜드코드'].isin(['5100,5338'])]
tmp = fac_product_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 공산품_포함관련 데이터 집계
fac_product_related = test_tr[test_tr['브랜드코드'].isin(['5100,5338'])]
tmp = fac_product_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 공산품_포함관련 구매 비율 계산
train_ft['공산품_포함관련구매비용비율'] = train_ft['공산품_포함관련구매비용'] / train_ft['총구매액']
train_ft['공산품_포함관련구매건수비율'] = train_ft['공산품_포함관련구매건수'] / train_ft['구매건수']

test_ft['공산품_포함관련구매비용비율'] = test_ft['공산품_포함관련구매비용'] / test_ft['총구매액']
test_ft['공산품_포함관련구매건수비율'] = test_ft['공산품_포함관련구매건수'] / test_ft['구매건수']

In [711]:
# # code test
# agg_list = [
#              ('', ),
#              ]
# train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()

In [712]:
# # code acting
# agg_list = [
#              ('', ),
#              ]

# tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [713]:
# tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

In [714]:
train_ft.shape, test_ft.shape

((14940, 348), (12225, 348))

## 중분류 활용

In [715]:
# 중분류별 구매횟수 평균mean 표준편차std 왜도skew 첨도kurt

In [716]:
# 기혼_특징
# 가전제품 : 가스기기, 가스렌지, 가전특정, 냉장고_세탁기, 냉장고.세탁기, TV_VTR, 냉난방
# 공산_식품 : 건강식품 건식품, 건어물, 조미료
# 생식품 : 곡물 생선 야채 정육 청과, 용기보증
# 여성_패션 : 가방 란제리 국내부띠끄 로얄부틱2F 명품 모피 모피_피혁 모피.피혁 모피_니트 미시케쥬얼 보석 부띠끄 수입부띠끄 수입종합화장품 엘레강스 엘레강스부틱 엘레강스캐주얼 타운웨어 크리스탈
# 남성_패션 : 트래디셔널 내셔날 남성라이센스
# 스포츠 : 골프(국내), 골프(수입), 골프(LC), 골프(NB), 골프 단품, 골프웨어

In [717]:
# 미혼_특징
# 패션 : 교복행사, 영캐주얼, 영캐쥬얼, 타운, 타운단품, 타운웨어, 화장품, 뉴베이직캐주얼, 베이직캐주얼, 시계, 트렌디캐주얼, 피혁토탈(B2),마담SIZE, 장신구
# 음식 : 일반조리, 즉석조리, 패스트푸드, 일용잡화
# 행사 : 단품_행사(캐릭터), 단품행사2
# 악기 : 영창(문화)
# 취미 : 취미소품

In [718]:
# train_tr['중분류'].nunique()

In [719]:
# all_middle_list = train_tr['중분류'].unique().tolist()
# len(all_middle_list)

In [720]:
# train_tr['구매가격'].sum()

In [721]:
# # ID별 모든중분류 구매가격
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index().head(30)

In [722]:
# # ID별 모든중분류 구매액 > ID별 중분류에 대한 총 구매가격
# train_tr.groupby(['ID'])['구매가격'].sum().reset_index()

In [723]:
# 중분류_기혼우세
elecs = ['가스기기', '가스렌지', '가전특정', '냉장고_세탁기', '냉장고.세탁기', 'TV_VTR', '냉난방']
fac_foods = ['건강식품', '건식품']
uncooked_foods = ['곡물', '생선', '야채', '정육', '청과']
fe_fashions = ['가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품', '모피', '모피_피혁', '모피.피혁,' '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어', '크리스탈']
ma_fashions = ['트래디셔널', '내셔날', '남성라이센스']
sports = ['골프(국내)', '골프(수입)', '골프(LC)', '골프(NB)', '골프단품', '골프웨어']
homewares = ['브랜드침구', '주방용품', '침구']

In [724]:
# # ID별 모든중분류에서 가전 제품에 해당되는 value가 있는지 확인
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index().groupby('ID')['중분류'].count()

In [725]:
# # 모든 중분류 구매 가격 총합
# price = []
# for all_middle in all_middle_list:
#   price.append(train_tr[train_tr['중분류'] == all_middle]['구매가격'].sum())

# sum(price)

In [726]:
# # 가전 제품 해당 value 총 금액
# elec_price = []
# for elec in elecs:
#   elec_price.append(train_tr[train_tr['중분류'] == elec ]['구매가격'].sum())

# sum(elec_price)

In [727]:
# # 공산_식품 해당 value 총 금액
# fac_price = []
# for fac_food in fac_foods:
#   fac_price.append(train_tr[train_tr['중분류'] == fac_food ]['구매가격'].sum())

# sum(fac_price)

In [728]:
# # 생식품 해당 value 총 금액
# uncooked_price = []
# for uncooked_food in uncooked_foods:
#   uncooked_price.append(train_tr[train_tr['중분류'] == uncooked_food ]['구매가격'].sum())

# sum(uncooked_price)

In [729]:
# # 여성패션 해당 value 총 금액
# fe_price = []
# for fe_fashion in fe_fashions:
#   fe_price.append(train_tr[train_tr['중분류'] == fe_fashion ]['구매가격'].sum())

# sum(uncooked_price)

In [730]:
# # 남성패션 해당 value 총 금액
# ma_price = []
# for ma_fashion in ma_fashions:
#   ma_price.append(train_tr[train_tr['중분류'] == ma_fashion ]['구매가격'].sum())

# sum(ma_price)

In [731]:
# # 스포츠 해당 value 총 금액
# sport_price = []
# for sport in sports:
#   sport_price.append(train_tr[train_tr['중분류'] == sport ]['구매가격'].sum())

# sum(sport_price)

In [732]:
# # 가정용품 해당 value 총 금액
# homeware_price = []
# for homeware in homewares:
#   homeware_price.append(train_tr[train_tr['중분류'] == homeware ]['구매가격'].sum())

# sum(homeware_price)

In [733]:
train_tr[train_tr['중분류'] == '용기보증']['대분류'].unique()

array(['공산품', '공산품파트', '생식품', '생식품파트', '케주얼_구두_아동', '영캐릭터', '스포츠캐쥬얼',
       '가정용품파트', '영라이브', '가정용품', '여성의류파트', '명품잡화', '영플라자', '남성정장스포츠',
       '여성캐주얼', '여성정장', '아동문화', '골프_유니캐쥬얼', '잡화', '아동_스포츠', '아동', '잡화파트',
       '여성캐쥬얼', '남성의류', '영어덜트캐쥬얼', '패션잡화', '로얄부틱', '로얄부띠끄'], dtype=object)

In [734]:
train_tr['중분류'] == '용기보증'

,중분류
0,False
1,False
2,True
3,False
4,False
...,...
523100,False
523101,False
523102,False
523103,True


In [735]:
# train_tr[train_tr['중분류'] == '가스기기']['구매가격'].sum() / train_tr['구매가격'].sum()

In [736]:
# train_tr[ x == '가스기기']['구매가격'].sum() / train_tr['구매가격'].sum()

In [737]:
# train_tr.groupby('ID')['중분류'].nunique()

In [738]:
# train_tr.groupby('ID')['중분류'].count()

# 기혼특징_중분류

In [739]:
# elecs = ['가스기기', '가스렌지', '가전특정', '냉장고_세탁기', '냉장고.세탁기', 'TV_VTR', '냉난방']
# fac_foods = ['건강식품', '건식품','조미료']
# uncooked_foods = ['곡물', '생선', '야채', '정육', '청과', '용기보증']
# fe_fashions = ['가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품', '모피', '모피_피혁', '모피.피혁,' '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어', '크리스탈']
# ma_fashions = ['트래디셔널', '내셔날', '남성라이센스']
# sports = ['골프(국내)', '골프(수입)', '골프(LC)', '골프(NB)', '골프단품', '골프웨어']
# middle_homewares = ['브랜드침구', '주방용품', '침구']
# > 구매비율

In [740]:
# 가전제품
agg_list = [
    ('가전제품관련구매비용', 'sum'),
    ('가전제품관련구매건수', 'count'),
    ('가전제품관련구매평균', 'mean')
]

# train 데이터에 대해 가전제품관련 데이터 집계
elecs_related = train_tr[train_tr['중분류'].str.contains('가스기기|가스렌지|가전특정|냉장고_세탁기|명품잡화|냉장고.세탁기|TV_VTR|냉난방', na=False)]
tmp = elecs_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 가전제품관련 데이터 집계
elecs_related = test_tr[test_tr['중분류'].str.contains('가스기기|가스렌지|가전특정|냉장고_세탁기|명품잡화|냉장고.세탁기|TV_VTR|냉난방', na=False)]
tmp = elecs_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 가전제품관련 구매 비율 계산
train_ft['가전제품관련구매비용비율'] = train_ft['가전제품관련구매비용'] / train_ft['총구매액']
train_ft['가전제품관련구매건수비율'] = train_ft['가전제품관련구매건수'] / train_ft['구매건수']

test_ft['가전제품관련구매비용비율'] = test_ft['가전제품관련구매비용'] / test_ft['총구매액']
test_ft['가전제품관련구매건수비율'] = test_ft['가전제품관련구매건수'] / test_ft['구매건수']

In [741]:
# 공산_식품
agg_list = [
    ('공산_식품관련구매비용', 'sum'),
    ('공산_식품관련구매건수', 'count'),
    ('공산_식품관련구매평균', 'mean')
]

# train 데이터에 대해 공산_식품관련 데이터 집계
fac_foods_related = train_tr[train_tr['중분류'].str.contains('건강식품|건식품|건어물|조미료', na=False)]
tmp = fac_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 공산_식품관련 데이터 집계
fac_foods_related = test_tr[test_tr['중분류'].str.contains('건강식품|건식품|건어물|조미료', na=False)]
tmp = fac_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 공산_식품관련 구매 비율 계산
train_ft['공산_식품관련구매비용비율'] = train_ft['공산_식품관련구매비용'] / train_ft['총구매액']
train_ft['공산_식품관련구매건수비율'] = train_ft['공산_식품관련구매건수'] / train_ft['구매건수']

test_ft['공산_식품관련구매비용비율'] = test_ft['공산_식품관련구매비용'] / test_ft['총구매액']
test_ft['공산_식품관련구매건수비율'] = test_ft['공산_식품관련구매건수'] / test_ft['구매건수']

In [742]:
# 생식품
agg_list = [
    ('생식품관련구매비용', 'sum'),
    ('생식품관련구매건수', 'count'),
    ('생식품관련구매평균', 'mean')
]

# train 데이터에 대해 생식품관련 데이터 집계
uncooked_foods_related = train_tr[train_tr['중분류'].str.contains('곡물|생선|야채|정육|청과|용기보증', na=False)]
tmp = uncooked_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 생식품관련 데이터 집계
uncooked_foods_related = test_tr[test_tr['중분류'].str.contains('곡물|생선|야채|정육|청과|용기보증', na=False)]
tmp = uncooked_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 생식품관련 구매 비율 계산
train_ft['생식품관련구매비용비율'] = train_ft['생식품관련구매비용'] / train_ft['총구매액']
train_ft['생식품관련구매건수비율'] = train_ft['생식품관련구매건수'] / train_ft['구매건수']

test_ft['생식품관련구매비용비율'] = test_ft['생식품관련구매비용'] / test_ft['총구매액']
test_ft['생식품관련구매건수비율'] = test_ft['생식품관련구매건수'] / test_ft['구매건수']

In [743]:
# 여성_패션
agg_list = [
    ('여성_패션관련구매비용', 'sum'),
    ('여성_패션관련구매건수', 'count'),
    ('여성_패션관련구매평균', 'mean')
]

# train 데이터에 대해 여성_패션관련 데이터 집계
fe_fashions_related = train_tr[train_tr['중분류'].str.contains('가방|란제리|국내부띠끄|로얄부틱2F|명품|모피|모피_피혁|모피.피혁|모피_니트|미시케쥬얼|보석|부띠끄|수입부띠끄|수입종합화장품|엘레강스|엘레강스부틱|엘레강스캐주얼|타운웨어|크리스탈', na=False)]
tmp = fe_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 여성_패션관련 데이터 집계
fe_fashions_related = test_tr[test_tr['중분류'].str.contains('가방|란제리|국내부띠끄|로얄부틱2F|명품|모피|모피_피혁|모피.피혁|모피_니트|미시케쥬얼|보석|부띠끄|수입부띠끄|수입종합화장품|엘레강스|엘레강스부틱|엘레강스캐주얼|타운웨어|크리스탈', na=False)]
tmp = fe_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 여성_패션관련 구매 비율 계산
train_ft['여성_패션관련구매비용비율'] = train_ft['여성_패션관련구매비용'] / train_ft['총구매액']
train_ft['여성_패션관련구매건수비율'] = train_ft['여성_패션관련구매건수'] / train_ft['구매건수']

test_ft['여성_패션관련구매비용비율'] = test_ft['여성_패션관련구매비용'] / test_ft['총구매액']
test_ft['여성_패션관련구매건수비율'] = test_ft['여성_패션관련구매건수'] / test_ft['구매건수']

In [744]:
# 남성_패션
agg_list = [
    ('남성_패션관련구매비용', 'sum'),
    ('남성_패션관련구매건수', 'count'),
    ('남성_패션관련구매평균', 'mean')
]

# train 데이터에 대해 남성_패션관련 데이터 집계
ma_fashions_related = train_tr[train_tr['중분류'].str.contains('트래디셔널|내셔날|남성라이센스', na=False)]
tmp = ma_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 남성_패션관련 데이터 집계
ma_fashions_related = test_tr[test_tr['중분류'].str.contains('트래디셔널|내셔날|남성라이센스', na=False)]
tmp = ma_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 남성_패션관련 구매 비율 계산
train_ft['남성_패션관련구매비용비율'] = train_ft['남성_패션관련구매비용'] / train_ft['총구매액']
train_ft['남성_패션관련구매건수비율'] = train_ft['남성_패션관련구매건수'] / train_ft['구매건수']

test_ft['남성_패션관련구매비용비율'] = test_ft['남성_패션관련구매비용'] / test_ft['총구매액']
test_ft['남성_패션관련구매건수비율'] = test_ft['남성_패션관련구매건수'] / test_ft['구매건수']

In [745]:
# 스포츠
agg_list = [
    ('스포츠관련구매비용', 'sum'),
    ('스포츠관련구매건수', 'count'),
    ('스포츠관련구매평균', 'mean')
]

# train 데이터에 대해 스포츠관련 데이터 집계
sports_related = train_tr[train_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]
tmp = sports_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 스포츠관련 데이터 집계
sports_related = test_tr[test_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]
tmp = sports_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 스포츠관련 구매 비율 계산
train_ft['스포츠관련구매비용비율'] = train_ft['스포츠관련구매비용'] / train_ft['총구매액']
train_ft['스포츠관련구매건수비율'] = train_ft['스포츠관련구매건수'] / train_ft['구매건수']

test_ft['스포츠관련구매비용비율'] = test_ft['스포츠관련구매비용'] / test_ft['총구매액']
test_ft['스포츠관련구매건수비율'] = test_ft['스포츠관련구매건수'] / test_ft['구매건수']

<ipython-input-745-ab2b1722fb8f>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sports_related = train_tr[train_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]
<ipython-input-745-ab2b1722fb8f>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sports_related = test_tr[test_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]


In [746]:
# 중_가정용품
agg_list = [
    ('중_가정용품관련구매비용', 'sum'),
    ('중_가정용품관련구매건수', 'count'),
    ('중_가정용품관련구매평균', 'mean')
]

# train 데이터에 대해 중_가정용품관련 데이터 집계
middle_homewares_related = train_tr[train_tr['중분류'].str.contains('브랜드침구|주방용품|침구', na=False)]
tmp = middle_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 중_가정용품관련 데이터 집계
middle_homewares_related = test_tr[test_tr['중분류'].str.contains('브랜드침구|주방용품|침구', na=False)]
tmp = middle_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 중_가정용품관련 구매 비율 계산
train_ft['중_가정용품관련구매비용비율'] = train_ft['중_가정용품관련구매비용'] / train_ft['총구매액']
train_ft['중_가정용품관련구매건수비율'] = train_ft['중_가정용품관련구매건수'] / train_ft['구매건수']

test_ft['중_가정용품관련구매비용비율'] = test_ft['중_가정용품관련구매비용'] / test_ft['총구매액']
test_ft['중_가정용품관련구매건수비율'] = test_ft['중_가정용품관련구매건수'] / test_ft['구매건수']

In [747]:
train_ft.shape, test_ft.shape

((14940, 383), (12225, 383))

# 미혼특징_중분류

In [748]:
# 미혼_특징
# 패션 : 교복행사, 영캐주얼, 영캐쥬얼, 타운, 타운단품, 타운웨어, 화장품, 뉴베이직캐주얼, 베이직캐주얼, 시계, 트렌디캐주얼, 피혁토탈(B2),마담SIZE, 장신구
# 음식 : 일반조리, 즉석조리, 패스트푸드
# 문화 : 단품행사2 영창(문화) 취미소품

In [749]:
# 패션
agg_list = [
    ('싱글패션관련구매비용', 'sum'),
    ('싱글패션관련구매건수', 'count'),
    ('싱글패션관련구매평균', 'mean')
]

# train 데이터에 대해 싱글패션관련 데이터 집계
single_fashions_related = train_tr[train_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]
tmp = single_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 싱글패션관련 데이터 집계
single_fashions_related = test_tr[test_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]
tmp = single_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 싱글패션관련 구매 비율 계산
train_ft['싱글패션관련구매비용비율'] = train_ft['싱글패션관련구매비용'] / train_ft['총구매액']
train_ft['싱글패션관련구매건수비율'] = train_ft['싱글패션관련구매건수'] / train_ft['구매건수']

test_ft['싱글패션관련구매비용비율'] = test_ft['싱글패션관련구매비용'] / test_ft['총구매액']
test_ft['싱글패션관련구매건수비율'] = test_ft['싱글패션관련구매건수'] / test_ft['구매건수']

<ipython-input-749-ded50f5babe8>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  single_fashions_related = train_tr[train_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]
<ipython-input-749-ded50f5babe8>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  single_fashions_related = test_tr[test_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]


In [750]:
# 음식
agg_list = [
    ('조리관련구매비용', 'sum'),
    ('조리관련구매건수', 'count'),
    ('조리관련구매평균', 'mean')
]

# train 데이터에 대해 조리관련 데이터 집계
cooked_food_related = train_tr[train_tr['중분류'].str.contains('일반조리|즉석조리|패스트푸드|일용잡화', na=False)]
tmp = cooked_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 조리관련 데이터 집계
cooked_food_related = test_tr[test_tr['중분류'].str.contains('일반조리|즉석조리|패스트푸드|일용잡화', na=False)]
tmp = cooked_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 조리관련 구매 비율 계산
train_ft['조리관련구매비용비율'] = train_ft['조리관련구매비용'] / train_ft['총구매액']
train_ft['조리관련구매건수비율'] = train_ft['조리관련구매건수'] / train_ft['구매건수']

test_ft['조리관련구매비용비율'] = test_ft['조리관련구매비용'] / test_ft['총구매액']
test_ft['조리관련구매건수비율'] = test_ft['조리관련구매건수'] / test_ft['구매건수']

In [751]:
# 문화
agg_list = [
    ('문화관련구매비용', 'sum'),
    ('문화관련구매건수', 'count'),
    ('문화관련구매평균', 'mean')
]

# train 데이터에 대해 문화관련 데이터 집계
cultures_related = train_tr[train_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]
tmp = cultures_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 문화관련 데이터 집계
cultures_related = test_tr[test_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]
tmp = cultures_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 문화관련 구매 비율 계산
train_ft['문화관련구매비용비율'] = train_ft['문화관련구매비용'] / train_ft['총구매액']
train_ft['문화관련구매건수비율'] = train_ft['문화관련구매건수'] / train_ft['구매건수']

test_ft['문화관련구매비용비율'] = test_ft['문화관련구매비용'] / test_ft['총구매액']
test_ft['문화관련구매건수비율'] = test_ft['문화관련구매건수'] / test_ft['구매건수']

<ipython-input-751-ef80547a1d55>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cultures_related = train_tr[train_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]
<ipython-input-751-ef80547a1d55>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cultures_related = test_tr[test_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]


In [752]:
fe_cols = '가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품 모피', '모피_피혁', '모피.피혁', '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어'
ma_cols = '트래디셔널', '내셔날', '남성라이센스'

In [753]:
# code_test
# agg_list = [
            # ('중분류_방문횟수_평균', lambda x: x.count() / x.nunique()),
            # ('가전기기_구매비율', train_tr.apply(lambda x: x['구매가격'] / train_tr['구매가격'].sum() if x['중분류'] == '가스기기' else 0, axis=1))
            # ('가전기기_구매비율', lambda x: train_tr[x == '가스기기']['구매가격'].sum() / train_tr['구매가격'].sum()),
            # ('공산_식품', lambda x: np.mean(x.str.endswith('식품').astype(int))),
            # ('생식품', lambda x: np.mean(x.isin(['곡물', '생선', '야채', '정육', '청과']).astype(int))),
            # ('여성_기혼_패션', lambda x: np.mean(x.isin(fe_cols).astype(int))),
            # ('남성_기혼_패션', lambda x: np.mean(x.isin(ma_cols).astype(int)))
            # ]

# train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index().head(60)

In [754]:
train_ft['총구매액']

,총구매액
0,4201200
1,5043025
2,22135386
3,35594762
4,3988866
...,...
14935,1195120
14936,2220000
14937,897898
14938,4136600


In [755]:
# # code_acting > 수정 및 보완 필요 부분
# agg_list = [
#             ('중분류_방문횟수_비율', lambda x: x.count() / x.nunique()),
#             ('가전제품', lambda x: np.mean(x.str.startswith('가').astype(int))),
#             ('공산_식품', lambda x: np.mean(x.str.endswith('식품').astype(int))),
#             ('생식품', lambda x: np.mean(x.isin(['곡물', '생선', '야채', '정육', '청과']).astype(int))),
#             ('여성_기혼_패션', lambda x: np.mean(x.isin(fe_cols).astype(int))),
#             ('남성_기혼_패션', lambda x: np.mean(x.isin(ma_cols).astype(int)))
#             ]

# tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [756]:
# tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [757]:
train_ft.shape, test_ft.shape

((14940, 398), (12225, 398))

##대분류 활용

In [758]:
# 기혼특징_대분류
#   가정 : 가정용품, 가정용품파트, 로얄부띠끄, 로얄부틱
#   골프 : 골프_유니캐주얼
#   명품 : 명품잡화, 로얄부띠끄, 로얄부틱
#   식품 : 생식품, 생식품파트
#   아동 : 아동, 아동_스포츠, 아동문화,
#   잡화 : 잡화, 잡화파트, 패션잡화

# 미혼특징_대분류
#   대_패션: 스포츠_캐주얼, 여성캐주얼, 여성캐쥬얼, 영라이브, 영어덜트캐쥬얼, 영캐릭터, 영플라자

In [759]:
# train_tr['대분류'].unique()

In [760]:
# train_tr['대분류'].isin(['가정용품','골프_유니캐주얼','생식품'])

In [761]:
# cols = ['가정용품', '골프_유니캐주얼', '로얄부띠끄', '로얄부틱', '명품잡화', '생식품']

In [762]:
# train_tr['대분류'].isin(cols).astype(int)

In [763]:
# (train_tr['대분류'].str.startswith('아','영'))

In [764]:
# train_tr['대분류'].str.endswith('아동').value_counts()

In [765]:
# ((train_tr['대분류'].str.startswith('아','영')) & (train_tr['대분류'].str.endswith('아동'))).astype(int)

In [766]:
# # code_test
# agg_list = [
#             ('아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int))),
#             ('여성_기혼', lambda x: np.mean((x.str.startswith('명품','로얄')).astype(int))),
#             ('남녀_기혼', lambda x: np.mean(x.isin(['가정용품','골프_유니캐주얼','생식품']))),


#             ]
# train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()

# 기혼특징_대분류

In [767]:
# 대_가정용품
agg_list = [
    ('대_가정용품관련구매비용', 'sum'),
    ('대_가정용품관련구매건수', 'count'),
    ('대_가정용품관련구매평균', 'mean')
]

# train 데이터에 대해 대_가정용품관련 데이터 집계
major_homewares_related = train_tr[train_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]
tmp = major_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 대_가정용품관련 데이터 집계
major_homewares_related = test_tr[test_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]
tmp = major_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 대_가정용품관련 구매 비율 계산
train_ft['대_가정용품관련구매비용비율'] = train_ft['대_가정용품관련구매비용'] / train_ft['총구매액']
train_ft['대_가정용품관련구매건수비율'] = train_ft['대_가정용품관련구매건수'] / train_ft['구매건수']

test_ft['대_가정용품관련구매비용비율'] = test_ft['대_가정용품관련구매비용'] / test_ft['총구매액']
test_ft['대_가정용품관련구매건수비율'] = test_ft['대_가정용품관련구매건수'] / test_ft['구매건수']

<ipython-input-767-e9b1e54d4ffc>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  major_homewares_related = train_tr[train_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]
<ipython-input-767-e9b1e54d4ffc>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  major_homewares_related = test_tr[test_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]


In [768]:
# 골프_유니
agg_list = [
    ('골프_유니관련구매비용', 'sum'),
    ('골프_유니관련구매건수', 'count'),
    ('골프_유니관련구매평균', 'mean')
]

# train 데이터에 대해 골프_유니관련 데이터 집계
golf_uni_related = train_tr[train_tr['대분류'].str.contains('골프_유니캐주얼', na=False)]
tmp = golf_uni_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 골프_유니관련 데이터 집계
golf_uni_related = test_tr[test_tr['대분류'].str.contains('골프_유니캐주얼', na=False)]
tmp = golf_uni_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 골프_유니관련 구매 비율 계산
train_ft['골프_유니관련구매비용비율'] = train_ft['골프_유니관련구매비용'] / train_ft['총구매액']
train_ft['골프_유니관련구매건수비율'] = train_ft['골프_유니관련구매건수'] / train_ft['구매건수']

test_ft['골프_유니관련구매비용비율'] = test_ft['골프_유니관련구매비용'] / test_ft['총구매액']
test_ft['골프_유니관련구매건수비율'] = test_ft['골프_유니관련구매건수'] / test_ft['구매건수']

In [769]:
# 명품
agg_list = [
    ('명품관련구매비용', 'sum'),
    ('명품관련구매건수', 'count'),
    ('명품관련구매평균', 'mean')
]

# train 데이터에 대해 명품관련 데이터 집계
luxury_related = train_tr[train_tr['대분류'].str.contains('명품잡화|로얄부띠끄|로얄부틱', na=False)]
tmp = luxury_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 명품관련 데이터 집계
luxury_related = test_tr[test_tr['대분류'].str.contains('명품잡화|로얄부띠끄|로얄부틱', na=False)]
tmp = luxury_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 명품관련 구매 비율 계산
train_ft['명품관련구매비용비율'] = train_ft['명품관련구매비용'] / train_ft['총구매액']
train_ft['명품관련구매건수비율'] = train_ft['명품관련구매건수'] / train_ft['구매건수']

test_ft['명품관련구매비용비율'] = test_ft['명품관련구매비용'] / test_ft['총구매액']
test_ft['명품관련구매건수비율'] = test_ft['명품관련구매건수'] / test_ft['구매건수']

In [770]:
# 대_식품
agg_list = [
    ('대_식품관련구매비용', 'sum'),
    ('대_식품관련구매건수', 'count'),
    ('대_식품관련구매평균', 'mean')
]

# train 데이터에 대해 대_식품관련 데이터 집계
major_food_related = train_tr[train_tr['대분류'].str.contains('생식품|생식품파트', na=False)]
tmp = major_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 대_식품관련 데이터 집계
major_food_related = test_tr[test_tr['대분류'].str.contains('생식품|생식품파트', na=False)]
tmp = major_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 대_식품관련 구매 비율 계산
train_ft['대_식품관련구매비용비율'] = train_ft['대_식품관련구매비용'] / train_ft['총구매액']
train_ft['대_식품관련구매건수비율'] = train_ft['대_식품관련구매건수'] / train_ft['구매건수']

test_ft['대_식품관련구매비용비율'] = test_ft['대_식품관련구매비용'] / test_ft['총구매액']
test_ft['대_식품관련구매건수비율'] = test_ft['대_식품관련구매건수'] / test_ft['구매건수']

In [771]:
# 아동
agg_list = [
    ('아동관련구매비용', 'sum'),
    ('아동관련구매건수', 'count'),
    ('아동관련구매평균', 'mean')
]

# train 데이터에 대해 아동관련 데이터 집계
child_related = train_tr[train_tr['대분류'].str.contains('아동|아동_스포츠|아동문화', na=False)]
tmp = child_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 아동관련 데이터 집계
child_related = test_tr[test_tr['대분류'].str.contains('아동|아동_스포츠|아동문화', na=False)]
tmp = child_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 아동관련 구매 비율 계산
train_ft['아동관련구매비용비율'] = train_ft['아동관련구매비용'] / train_ft['총구매액']
train_ft['아동관련구매건수비율'] = train_ft['아동관련구매건수'] / train_ft['구매건수']

test_ft['아동관련구매비용비율'] = test_ft['아동관련구매비용'] / test_ft['총구매액']
test_ft['아동관련구매건수비율'] = test_ft['아동관련구매건수'] / test_ft['구매건수']

In [772]:
# 잡화
agg_list = [
    ('잡화관련구매비용', 'sum'),
    ('잡화관련구매건수', 'count'),
    ('잡화관련구매평균', 'mean')
]

# train 데이터에 대해 잡화관련 데이터 집계
sundry_related = train_tr[train_tr['대분류'].str.contains('잡화|잡화파트|패션잡화', na=False)]
tmp = sundry_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 잡화관련 데이터 집계
sundry_related = test_tr[test_tr['대분류'].str.contains('잡화|잡화파트|패션잡화', na=False)]
tmp = sundry_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 잡화관련 구매 비율 계산
train_ft['잡화관련구매비용비율'] = train_ft['잡화관련구매비용'] / train_ft['총구매액']
train_ft['잡화관련구매건수비율'] = train_ft['잡화관련구매건수'] / train_ft['구매건수']

test_ft['잡화관련구매비용비율'] = test_ft['잡화관련구매비용'] / test_ft['총구매액']
test_ft['잡화관련구매건수비율'] = test_ft['잡화관련구매건수'] / test_ft['구매건수']

In [773]:
train_ft.shape, test_ft.shape

((14940, 428), (12225, 428))

# 미혼특징_대분류

In [774]:
# 대_패션
agg_list = [
    ('대_패션관련구매비용', 'sum'),
    ('대_패션관련구매건수', 'count'),
    ('대_패션관련구매평균', 'mean')
]

# train 데이터에 대해 대_패션관련 데이터 집계
major_fashion_related = train_tr[train_tr['대분류'].str.contains('스포츠_캐주얼|여성캐주얼|여성캐쥬얼|영라이브|영어덜트캐쥬얼|영캐릭터|영플라자', na=False)]
tmp = major_fashion_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 대_패션관련 데이터 집계
major_fashion_related = test_tr[test_tr['대분류'].str.contains('스포츠_캐주얼|여성캐주얼|여성캐쥬얼|영라이브|영어덜트캐쥬얼|영캐릭터|영플라자', na=False)]
tmp = major_fashion_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 대_패션관련 구매 비율 계산
train_ft['대_패션관련구매비용비율'] = train_ft['대_패션관련구매비용'] / train_ft['총구매액']
train_ft['대_패션관련구매건수비율'] = train_ft['대_패션관련구매건수'] / train_ft['구매건수']

test_ft['대_패션관련구매비용비율'] = test_ft['대_패션관련구매비용'] / test_ft['총구매액']
test_ft['대_패션관련구매건수비율'] = test_ft['대_패션관련구매건수'] / test_ft['구매건수']

In [775]:
# # code acting
# agg_list = [
#             ('아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int))),
#             ('여성_기혼', lambda x: np.mean((x.str.startswith('명품','로얄')).astype(int))),
#             ('남녀_기혼', lambda x: np.mean(x.isin(['가정용품','골프_유니캐주얼','생식품']))),
#             ('영_아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int)))
#             ]

# tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [776]:
# tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [777]:
train_ft.shape, test_ft.shape

((14940, 433), (12225, 433))

## 구매가격 활용

In [778]:
# 구매가격 사분위수, 전체 고객 VIP 10%
# 명품+가전제품의 구매가격 상위 10%, 패션+화장품의 구매가격 상위 10%

In [779]:
train_tr.isnull().sum()

,0
ID,0
구매일시,0
지점코드,0
대분류,0
중분류,0
브랜드코드,0
구매가격,0
hour,0
date,0
18시~21시,0


In [780]:
# 사분위수 계산 (0.25, 0.75)
agg_list = [
    ('구매가격_사분위수_Q1', lambda x: x.quantile(0.25)),
    ('구매가격_사분위수_Q3', lambda x: x.quantile(0.75))
]

# train 데이터에 대해 사분위수관련 데이터 집계
tmp = train_tr.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 사분위수관련 데이터 집계
tmp = test_tr.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 각 ID별로 전체 구매가격의 상위 10% 계산
train_top10_threshold = train_tr.groupby('ID')['구매가격'].apply(lambda x: x.nlargest(int(len(x) * 0.1)).min()).reset_index()
train_top10_threshold.columns = ['ID', '상위10%_구매가격']  # 컬럼명 변경
train_ft = train_ft.merge(train_top10_threshold, how='left', on='ID')

test_top10_threshold = test_tr.groupby('ID')['구매가격'].apply(lambda x: x.nlargest(int(len(x) * 0.1)).min()).reset_index()
test_top10_threshold.columns = ['ID', '상위10%_구매가격']  # 컬럼명 변경
test_ft = test_ft.merge(test_top10_threshold, how='left', on='ID')

# 각 ID별 구매가격의 합계 계산
train_sum = train_tr.groupby('ID')['구매가격'].sum().reset_index()
train_sum.columns = ['ID', '구매가격_합계']
train_ft = train_ft.merge(train_sum, how='left', on='ID')

test_sum = test_tr.groupby('ID')['구매가격'].sum().reset_index()
test_sum.columns = ['ID', '구매가격_합계']
test_ft = test_ft.merge(test_sum, how='left', on='ID')

# 상위 10% 여부 판별
train_ft['상위10%_여부'] = (train_ft['구매가격_합계'] >= train_ft['상위10%_구매가격']).astype(int)
test_ft['상위10%_여부'] = (test_ft['구매가격_합계'] >= test_ft['상위10%_구매가격']).astype(int)

# # 상위 10% 추출
# top10_percent = train_ft.nlargest(int(len(train_ft) * 0.1), '사분위수관련구매비용비율')
# train_ft['VIP_grade'] =  top10_percent

In [781]:
train_ft['상위10%_구매가격'].isnull().sum()

3652

In [782]:
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,대_패션관련구매비용,대_패션관련구매건수,대_패션관련구매평균,대_패션관련구매비용비율,대_패션관련구매건수비율,구매가격_사분위수_Q1,구매가격_사분위수_Q3,상위10%_구매가격,구매가격_합계,상위10%_여부
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,NaN,NaN,NaN,NaN,NaN,41375.0,362500.0,1076000.0,4201200,1
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,3426200.0,21.0,163152.380952,0.679394,0.500000,37500.0,146000.0,300000.0,5043025,1
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,1003000.0,5.0,200600.000000,0.045312,0.043860,37514.0,189750.0,396000.0,22135386,1
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,3462800.0,17.0,203694.117647,0.097284,0.080569,49000.0,200500.0,456000.0,35594762,1
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,NaN,NaN,NaN,NaN,NaN,12211.0,223750.0,695000.0,3988866,1


In [783]:
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,대_패션관련구매비용,대_패션관련구매건수,대_패션관련구매평균,대_패션관련구매비용비율,대_패션관련구매건수비율,구매가격_사분위수_Q1,구매가격_사분위수_Q3,상위10%_구매가격,구매가격_합계,상위10%_여부
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,100000.0,1.0,100000.000000,0.232786,0.142857,37200.0,80000.0,NaN,429579,0
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,62000.0,1.0,62000.000000,0.213352,0.250000,55900.0,84250.0,NaN,290600,0
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,NaN,NaN,NaN,NaN,NaN,52000.0,96000.0,NaN,607000,0
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,1144200.0,17.0,67305.882353,0.496240,0.531250,18400.0,153250.0,380000.0,2305740,1
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,1257200.0,10.0,125720.000000,0.801433,0.476190,27000.0,125100.0,270000.0,1568690,1


In [ ]:
# # code_test
# agg_list = [
#         ('vip', lambda x: pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num]),
#         ('사분위수', 'quantile')
#     ]

# train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()

In [ ]:
# # code_acting
# vip_num = int(train_tr.groupby('ID')['구매가격'].mean().count() * 0.1)
# agg_list = [
#         ('vip', lambda x: pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num]),
#     ]

# tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [ ]:
# tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [784]:
train_ft.shape, test_ft.shape

((14940, 438), (12225, 438))

In [ ]:
# train_tr['중분류'].unique()

In [ ]:
# train_tr['대분류'].unique()

In [ ]:
# train_tr[train_tr['대분류'] == '']['중분류'].unique()

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [785]:
train_ft.shape, test_ft.shape

((14940, 438), (12225, 438))

In [786]:
train_ft = train_ft.fillna(0)
test_ft = test_ft.fillna(0)

train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [787]:
train_ft.shape, test_ft.shape

((14940, 438), (12225, 438))

In [788]:
train_ft = train_ft.drop(columns="ID")
test_ft = test_ft.drop(columns="ID")

In [789]:
cols = train_ft.select_dtypes("object").columns.tolist()
cols

['주구매지점', '주구매_중분류', '주구매_대분류']

In [790]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown="ignore")

In [791]:
enc.fit_transform(train_ft[cols]).toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [792]:
tmp = pd.DataFrame(
    enc.fit_transform(train_ft[cols]).toarray(),
    columns =  enc.get_feature_names_out()
)

train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [793]:
test_ft.columns

Index(['내점일수', '구매주기', '주말방문비율', '봄_구매비율', '여름_구매비율', '가을_구매비율', '겨울_구매비율',
       '주구매요일', '일별평균구매건수', '거래개월수',
       ...
       '대_패션관련구매비용', '대_패션관련구매건수', '대_패션관련구매평균', '대_패션관련구매비용비율',
       '대_패션관련구매건수비율', '구매가격_사분위수_Q1', '구매가격_사분위수_Q3', '상위10%_구매가격', '구매가격_합계',
       '상위10%_여부'],
      dtype='object', length=437)

In [794]:
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)
test_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [795]:
train_ft.shape, test_ft.shape

((14940, 712), (12225, 712))

In [796]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [797]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,-0.437126,0.002987,0.257728,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269
1,0.239394,-0.356452,-1.008554,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,4.254981,-0.228606,-0.342432,-0.280675,-0.319269
2,1.890101,-0.869935,0.036742,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269
3,4.298510,-1.023980,-0.080558,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,3.562844,-0.319269
4,0.618244,-0.613193,0.302875,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269


In [798]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [799]:
from sklearn.model_selection import cross_val_score, KFold
cv = KFold(5, shuffle = True, random_state = 42)

In [800]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=42)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [801]:
scores = cross_val_score(model, train_ft, target, cv = cv, scoring = 'f1_macro', n_jobs = -1)
scores.mean()

0.7142982235740483

In [802]:
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057781 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 27133
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 528
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

In [803]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [804]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,0.0
4,test_4,0.0
...,...,...
12220,test_12220,0.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


# 추출한 피처 저장하기

In [805]:
final_project = '/content/drive/MyDrive/멋사_final_project/'

In [806]:
# train_ft.to_csv(f"{final_project}train_common_DK_2024_10_30_1차.csv",index=False)
# test_ft.to_csv(f"{final_project}test_common_DK_2024_10_30_1차.csv",index=False)

In [807]:
submit.to_csv(f"{final_project}DK_2024_11_03_4차_submit.csv",index=False)